The times taken to query the base galaxy Table using obsMetadata and instance Catalogs is much larger than the time to query the galaxy table on fatboy directly. This records timeit results for these queries

## Setup

In [9]:
from __future__ import division, print_function, absolute_import

In [13]:
import time
import os
import pandas as pd
import numpy as np

In [2]:
from lsst.sims.catalogs.measures.instance import InstanceCatalog
import lsst.sims.catUtils.baseCatalogModels as bcm
from lsst.sims.catalogs.generation.db import CatalogDBObject, DBObject
from lsst.sims.catUtils.baseCatalogModels.GalaxyModels import GalaxyTileObj, GalaxyObj
from lsst.sims.utils import ObservationMetaData

/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


## galaxy Instance Catalog + writeCatalog

In [38]:
observerMetaData = ObservationMetaData(boundType='box',
                                  pointingRA=0.,
                                  pointingDec=0.,
                                  boundLength=2.)

In [40]:
observerMetaData.boundLength

2.0

In [55]:
def tileObsMetaData(obsMetaData, num):
    ra, dec, size = obsMetaData.pointingRA, obsMetaData.pointingDec, obsMetaData.boundLength
    blen = size / num
    pras = np.arange(ra - size, ra + size, 2.0 *blen) + blen
    pdecs = np.arange(dec - size, dec + size, 2.0 * blen) + blen
    obsmdata = [ObservationMetaData(boundType='box',
                                  pointingRA=pras[i],
                                  pointingDec=pdecs[i],
                                  boundLength=blen) for i in range(len(pras))]
    return obsmdata

tiledObsMD = tileObsMetaData(observerMetaData, 20)

In [56]:
tiledObsMD

In [13]:
class galBaseCopy(InstanceCatalog):
    column_outputs = ['id', 'raJ2000', 'decJ2000', 'redshift']#, 'a_d', 'b_d', 'pa_disk', 'mass_stellar', 'absmag_r_total']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'a_d': '%8e', 'b_d': '%8e', 'pa_disk': '%8e', 
                        'mass_stellar': '%8e', 'absmag_r_total': '%8e'}

In [51]:
class galCopy(InstanceCatalog):
    column_outputs = ['galtileid', 'raJ2000', 'decJ2000', 'redshift']#, 'a_d', 'b_d', 'pa_disk', 'mass_stellar', 'absmag_r_total']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'a_d': '%8e', 'b_d': '%8e', 'pa_disk': '%8e', 
                        'mass_stellar': '%8e', 'absmag_r_total': '%8e'}

In [59]:
def writeCat(obsMD=tiledObsMD, redshiftmax=0.4, fname='gB-test.csv', chunkSize=None):
    constr = 'redshift < {0}'.format(redshiftmax)
    galaxyTiled = GalaxyTileObj()
    print(len(obsMD))
    for i, obsMetaData in enumerate(obsMD):
        filename = fname  + '{}'.format(i)
        galaxyBase = galCopy(galaxyTiled, obs_metadata=obsMetaData, constraint=constr)
        galaxyBase.write_catalog(filename, chunkSize)
            

In [57]:
def writeCatBase(redshiftmax=0.4, fname='gB-test.csv', chunkSize=None):
    constr = 'redshift < {0}'.format(redshiftmax)
    galaxyTiled = GalaxyObj()
    galaxyBase = galBaseCopy(galaxyTiled, obs_metadata=obsMetaData, constraint=constr)
    galaxyBase.write_catalog(fname, chunkSize)

In [60]:
%%timeit
chunksize = None
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

20
123.287425995
20
123.117075205
20
131.047254086
20
120.867078066
1 loops, best of 3: 2min per loop


In [61]:
!wc -l *.csv

   35307 gB-test.csv
  907563 gb_pd.test.csv
    4952 results.csv
  947822 total


In [51]:
def writeDBObjBase(redshiftmax=0.4, fname='gB-test.csv', chunkSize=None):
    constr = 'redshift < {0}'.format(redshiftmax)
    galaxyBase = GalaxyTileObj()
    # galDB = dbobject = DBObject(database=config.database, driver='mssql+pymssql', host=config.host, port=config.port, connection=None)
    #galaxyBase = galBaseCopy(galaxyTiled, obs_metadata=obsMetaData, constraint=constr)
    query = """SELECT galtileid, raJ2000, decJ2000, redshift FROM galaxy WHERE redshift < 0.4 AND raJ2000 < 2 and raJ2000 > -2 and decJ2000 < 2 and decJ2000 > -2"""
    x = galaxyBase.execute_arbitrary(query)
    print(len(x), len(x[0]))
    #galaxyBase.write_catalog(fname, chunkSize)

In [53]:
def writeDBObjBase(redshiftmax=0.4, fname='gB-test.csv', chunkSize=None):
    constr = 'redshift < {0}'.format(redshiftmax)
    galaxyBase = GalaxyObj()
    # galDB = dbobject = DBObject(database=config.database, driver='mssql+pymssql', host=config.host, port=config.port, connection=None)
    #galaxyBase = galBaseCopy(galaxyTiled, obs_metadata=obsMetaData, constraint=constr)
    query = """SELECT id, ra, dec, redshift WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
    x = galaxyBase.execute_arbitrary(query)
    print(len(x), len(x[0]))
    #galaxyBase.write_catalog(fname, chunkSize)

In [54]:
%%timeit
chunksize = None
tstart = time.time()
writeDBObjBase(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

ProgrammingError: (ProgrammingError) (207, "Invalid column name 'redshift'.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the S

In [25]:
%%timeit
chunksize = None
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

TypeError: writeCat() takes at least 1 argument (1 given)

In [8]:
%%timeit
chunksize = 50000
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

52.747150898
50.3756668568
52.9057190418
50.5252020359
1 loops, best of 3: 50.4 s per loop


In [9]:
%%timeit
chunksize = 500000
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

51.8218009472
49.6426441669
52.1382918358
55.8443951607
1 loops, best of 3: 49.6 s per loop


In [10]:
%%timeit
chunksize = 5000
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

52.6447761059
52.5556430817
50.3972399235
49.5057430267
1 loops, best of 3: 49.5 s per loop


In [15]:
%%timeit
chunksize = 50
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

54.0655651093
54.7553839684
55.7499918938
56.8625900745
1 loops, best of 3: 54.8 s per loop


## Direct query with fetch

In [20]:
from lsst.utils import getPackageDir
from lsst.daf.persistence import DbAuth
import lsst.pex.config as pexConfig
import pymssql

In [4]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
x = db.fetchall()
print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

(907562, 4)
2.89684391022
(907562, 4)
2.78780198097
(907562, 4)
2.91004490852
(907562, 4)
2.87404108047
1 loops, best of 3: 2.79 s per loop


In [9]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
num = 50000
x = db.fetchmany(num)
while len(x)>0:
    x = db.fetchmany(num)
# print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

47.9792869091
23.2235229015
34.0013780594
37.0588347912
1 loops, best of 3: 23.2 s per loop


In [7]:
!pwd

/Users/rbiswas/src/LSST/BenchmarkInstanceCatalogs/examples


In [10]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
#num = 50000
x = db.fetchall()
# print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

38.0828609467
48.4139699936
46.8077619076
42.6143682003
1 loops, best of 3: 42.6 s per loop


In [16]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
num = 50000
x = db.fetchmany(num)
while len(x) > 0:
    x = db.fetchmany(num)
# print(len(x), len(x[0]))
#df = pd.DataFrame.from_records(x, columns=['id', 'ra', 'dec', 'redshift'])
#df.to_csv('gb_pd.test.csv', index=False)
tend = time.time()
print(tend - tstart)

3.06677079201
2.84902787209
2.82190299034
2.79049491882
1 loops, best of 3: 2.79 s per loop


In [14]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
#num = 50000
x = db.fetchall()
# print(len(x), len(x[0]))
df = pd.DataFrame.from_records(x, columns=['id', 'ra', 'dec', 'redshift'])
df.to_csv('gb_pd.test.csv', index=False)
tend = time.time()
print(tend - tstart)

5.25460505486
5.22433400154
5.32429909706
5.30123806
1 loops, best of 3: 5.22 s per loop


In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
#num = 50000
x = db.fetchall()
# print(len(x), len(x[0]))
df = pd.DataFrame.from_records(x, columns=['id', 'ra', 'dec', 'redshift'])
df.to_csv('gb_pd.test.csv', index=False)
tend = time.time()
print(tend - tstart)

In [23]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT galaxy.id AS id, ra*PI()/180. AS [raJ2000], dec*PI()/180. AS [decJ2000], galaxy.redshift AS redshift 
FROM galaxy  WITH(FORCESEEK) JOIN (SELECT htmidstart, htmidend 
FROM fHtmCoverRegion(%(fHtmCoverRegion_1)s)) AS anon_1 ON galaxy.htmid BETWEEN anon_1.htmidstart AND anon_1.htmidend 
WHERE sph.fRegionContainsXYZ(sph.fSimplifyString(%(fSimplifyString_1)s), galaxy.cx, galaxy.cy, galaxy.cz) = %(fRegionContainsXYZ_1)s
"""
db.execute(query)
#num = 50000
x = db.fetchall()
# print(len(x), len(x[0]))
df = pd.DataFrame.from_records(x, columns=['id', 'ra', 'dec', 'redshift'])
df.to_csv('gb_pd.test.csv', index=False)
tend = time.time()
print(tend - tstart)

ProgrammingError: (102, "Incorrect syntax near '('.DB-Lib error message 20018, severity 15:\nGeneral SQL Server error: Check messages from the SQL Server\n")

In [26]:
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)



In [35]:
config

lsst.sims.catUtils.baseCatalogModels.BaseCatalogModels.BaseCatalogConfig(host='localhost', driver='mssql+pymssql', port='51433', database='LSSTCATSIM')

In [36]:
dbobject = DBObject(database=config.database, driver='mssql+pymssql', host=config.host, port=config.port, connection=None)

In [ ]:
dbobject.execute_arbitrary()